### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [1]:
import numpy as np

### generate a simple linear dataset y=xb+e

In [2]:
import data_gen as dg
data1=dg.data_gen(data_shape=(400,3),seed=0)
y=data1.y
x=data1.x[:,1:] #drop first column of x (all 1's)
e=data1.e
n,p=x.shape

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
 - 'product'-Ndiff1 multiplied by Ndiff2
 - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
 - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
 - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- kern_grid: 
 - no means smooth the original data
 - int means form a grid across all the variables for xin, yxin with int evenly spaced 
 observations (could make flexible than uniform distribution)
- Ndiff_bw_kern:
 - rbfkern means use the radial basis function kernel
 - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
 - 'NW' means use nadaraya-watson kernel regression
 - 'full_logit' means local logit with all variables entering linearly
 - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- kern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data, which is useful for calibrating hyper parameters
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random 
variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels 
  across that random variable (as if creating a cdf)
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - p_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [3]:
max_bw_Ndiff=2
modeldict1={
    'Ndiff_type':'product',
    'max_bw_Ndiff':max_bw_Ndiff,
    'normalize_Ndiffwtsum':'own_n',
    'kern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'none',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'p_bandscale':'non-neg',
        'Ndiff_depth_bw':'non-neg',
        'outer_x_bw':'non-neg',
        'outer_y_bw':'non-neg'
        }
    }

#set starting or fixed values for hyper parameters
hyper_paramdict1={
    'Ndiff_exponent':.05*np.ones([max_bw_Ndiff-1,]),
    'p_bandscale':np.ones([p,]),
    'outer_x_bw':np.array([0.5,]),
    'outer_y_bw':np.array([0.5,]),
    'Ndiff_depth_bw':np.ones([max_bw_Ndiff,])*0.15
    }
#assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
#                                                                            'depth of max_bw_Ndiff'
assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
                                                                             'depth of max_bw_Ndiff'
#create optimization dictionary
optimizedict1={
    'method':'Nelder-Mead',
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

#-----------------Calibrate/Optimize--------

In [4]:
import mykern as mk
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)

opitmization starting
excess numerator mask count 3160
excess denom mask count 0
gkernh maskcount 3160 kernshape (40, 40, 40) maskeddata_maskcount 3160 maskeddata_shape (40, 40, 40) hmaskcount 0
excess numerator mask count 3160
excess denom mask count 0
gkernh maskcount 3160 kernshape (40, 40, 40) maskeddata_maskcount 3160 maskeddata_shape (40, 40, 40) hmaskcount 0
40 xbw masked count 0 xonediffs mask count
80 yx_bw_endstack masked count 0 yonediffs mask count
excess numerator mask count 0
excess denom mask count 0
gkernh maskcount 40 kernshape (40, 40) maskeddata_maskcount 0 maskeddata_shape (40, 40) hmaskcount 40
40 alkerns masked count with shape (40, 40)
excess numerator mask count 0
excess denom mask count 0
gkernh maskcount 80 kernshape (40, 40, 2) maskeddata_maskcount 0 maskeddata_shape (40, 40, 2) hmaskcount 80
80 alkerns masked count with shape (40, 40, 2)
40 are masked in cdfnorm_prob_yx of shape: (40, 40)
40 are masked in cdfnorm_prob_x of shape: (40, 40)
mse: 13260.55600127

In [5]:
print(optimized_Ndiff_kernel.mse)
import myreg
just_linreg=myreg.reg(y,x)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

 final_simplex: (array([[ 0.06211   , -0.4984641 ,  0.44789289,  1.76409231,  0.40338112,
         0.04276216, -0.42123349,  2.11672633],
       [ 0.0621093 , -0.49845059,  0.44789711,  1.76408415,  0.40337932,
         0.04276553, -0.4212368 ,  2.11672971],
       [ 0.06210941, -0.49845464,  0.44789655,  1.76408323,  0.40337964,
         0.04276578, -0.42124057,  2.11673253],
       [ 0.06210979, -0.49846421,  0.44789281,  1.76408866,  0.40338126,
         0.04276414, -0.42124461,  2.11673565],
       [ 0.06210919, -0.49845605,  0.44789775,  1.76408377,  0.40338034,
         0.04276532, -0.42124239,  2.11673766],
       [ 0.06210897, -0.49845924,  0.44789654,  1.76409232,  0.40338247,
         0.042763  , -0.42124339,  2.11674641],
       [ 0.06210908, -0.49846426,  0.44789617,  1.76409478,  0.40338325,
         0.042762  , -0.42124426,  2.11674805],
       [ 0.06210878, -0.49846389,  0.44789791,  1.76409826,  0.40338424,
         0.0427604 , -0.42124283,  2.11675155],
       [ 0.0621

In [6]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [7]:
xgrid=np.linspace(1,n,n)
yhat=optimized_Ndiff_kernel.yhat
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=5,color='blue',alpha=0.6,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',legend='yhat')
#p.line(xgrid,gageht,color='green',legend='gage height')
p.legend.location = "top_left"
p.yaxis.visible=False
show(p)

In [8]:
yhat.shape

(40,)

In [9]:
print(yhat)

[-29.463333386097318 0.10295805804338443 -32.03672715290523
 -2.774022112520546 -1.0041786509308857 21.103946940442718
 4.317350747725802 -34.705941077260604 -16.80270853599206
 -33.73664303961993 1.0827998967171624 -12.187794178370684
 -15.817729537453523 -20.063989009724725 10.610311157943032
 -20.090884320404395 -26.10884401577198 30.55732049156923
 -23.4272375059564 -12.567125387954215 -25.899092545249793
 3.0478366170122744 -7.015644801558554 -4.51292432751867
 -46.73092152738743 -33.97094106902052 13.964585625223206
 -7.826784505885228 -20.555351638337967 6.028366889303493
 8.951273901857567 -29.017746226701966 -20.749806708600048
 -7.944311538820161 -27.207823555698493 -18.930733394187925
 -13.696817985856125 -28.91231108404761 -42.46500800007052
 -19.640937307378916]


In [10]:
print(y)

[-26.97958545  -2.43761463 -30.9591471   -1.32628344  -3.6715343
  14.52129391   3.0090395  -31.76919722 -16.63533493 -32.52834824
  20.66999742 -12.70059055 -15.66107612 -19.199295    10.48539265
 -64.80113608 -27.00057232  21.79971341 -22.67183098 -12.95259814
 -24.01069784  -0.42785548  -8.51551739  -6.25480112 -44.25922483
 -35.40924103  19.5048316   -8.64763895 -20.22814672   7.54315657
  23.09839021 -26.2721089  -70.67285875  -9.10671098 -25.43473326
 -18.15889806 -13.92271607 -26.52634001 -37.51406772 -19.36715735]


In [11]:
optimized_Ndiff_kernel.fixed_or_free_paramdict

{'Ndiff_exponent': {'fixed_or_free': 'free',
  'const': 'free',
  'location_idx': (0, 1)},
 'p_bandscale': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (1, 4)},
 'Ndiff_depth_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (4, 6)},
 'outer_x_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (6, 7)},
 'outer_y_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (7, 8)},
 'free_params': array([ 0.06210919, -0.49845605,  0.44789775,  1.76408377,  0.40338034,
         0.04276532, -0.42124239,  2.11673766]),
 'fixed_params': array([], dtype=float64)}

In [12]:
np.ma.count_masked(yhat)

0